# Segment Customers by Recency, Frequency, and Monetary Value (RFM)
This code was my attempt to follow along Susan Li's wonderful RFM analysis titled: Find Your Best Customers with Customer Segmentation in Python.

I visualize this analysis further on Tableau Public: 

In [1]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime

In [2]:
warnings.filterwarnings('ignore')

In [3]:
#dataset is the Market Basket Analysis - Online retail dataset from UCI ML Repository
retail = pd.read_csv('OnlineRetail.csv', encoding = 'unicode_escape')
retail.head()
df = retail


In [4]:
#Unique Variables
print(df.Country.nunique())
print(df.Country.unique())

38
['United Kingdom' 'France' 'Australia' 'Netherlands' 'Germany' 'Norway'
 'EIRE' 'Switzerland' 'Spain' 'Poland' 'Portugal' 'Italy' 'Belgium'
 'Lithuania' 'Japan' 'Iceland' 'Channel Islands' 'Denmark' 'Cyprus'
 'Sweden' 'Austria' 'Israel' 'Finland' 'Bahrain' 'Greece' 'Hong Kong'
 'Singapore' 'Lebanon' 'United Arab Emirates' 'Saudi Arabia'
 'Czech Republic' 'Canada' 'Unspecified' 'Brazil' 'USA'
 'European Community' 'Malta' 'RSA']


In [5]:
#Drop Duplicate customer IDs per Country
U_Customer_Country = df[['Country', 'CustomerID']].drop_duplicates()

#Display Count of unique customers per Country
U_Customer_Country.groupby('Country')['CustomerID'].aggregate('count').reset_index().sort_values('CustomerID', ascending=False).head(10)

,Country,CustomerID
36,United Kingdom,3950
14,Germany,95
13,France,87
31,Spain,31
3,Belgium,25
33,Switzerland,21
27,Portugal,19
19,Italy,15
12,Finland,12
1,Austria,11


In [6]:
#Dataset will only focus on the top four countries
Countries = ['United Kingdom','Germany','France','Spain']
df = df[df.Country.isin(Countries)]

In [7]:
#How many are blank?
df.isnull().sum(axis=0)

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     133666
Country             0
dtype: int64

In [8]:
df = df[pd.notnull(df['CustomerID'])]
df['CustomerID'] = df['CustomerID'].astype(int)
df['CustomerID'] = df['CustomerID'].astype(str)

In [9]:
print('The minimum value is:' + str(df.Quantity.min()))

The minimum value is:-80995


In [10]:
#Remove negative values in Quantity
df = df[(df['Quantity']>0)]
print(df.shape)
print(df.info())

(374214, 8)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 374214 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      374214 non-null object
StockCode      374214 non-null object
Description    374214 non-null object
Quantity       374214 non-null int64
InvoiceDate    374214 non-null object
UnitPrice      374214 non-null float64
CustomerID     374214 non-null object
Country        374214 non-null object
dtypes: float64(1), int64(1), object(6)
memory usage: 25.7+ MB
None


In [11]:
#Check for unique values in each column
def uniques(df):
    for i in df.columns:
        count = df[i].nunique()
        print(i,":", count)

uniques(df)

InvoiceNo : 17585
StockCode : 3648
Description : 3847
Quantity : 295
InvoiceDate : 16448
UnitPrice : 417
CustomerID : 4132
Country : 4


In [12]:
#Calculate Total Price
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

In [13]:
#Calculate Dates
now = datetime(2011,12,31)
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
print('Earliest date is:', df['InvoiceDate'].min())
print('LookupErroratest date is:', df['InvoiceDate'].max())

Earliest date is: 2010-12-01 08:26:00
LookupErroratest date is: 2011-12-09 12:50:00


In [14]:
#RFM Segmentation
RFMtable = df.groupby('CustomerID').agg({'InvoiceDate': lambda x: (now - x.max()).days,
                                        'InvoiceNo': lambda x: len(x), 'TotalPrice': lambda x: x.sum()})

RFMtable['InvoiceDate'] = RFMtable['InvoiceDate'].astype(int)
RFMtable.rename(columns={'InvoiceDate': 'recency', 
                        'InvoiceNo': 'frequency',
                        'TotalPrice': 'monetary_value'},
               inplace=True)

In [15]:
RFMtable.head()

,recency,frequency,monetary_value
CustomerID,,,
12346,346,1,77183.60
12354,253,58,1079.40
12413,87,38,758.10
12417,246,23,436.30
12421,36,45,807.04


In [16]:
#Check a Customer
first_customer = df[df['CustomerID'] == '12346']
first_customer

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346,United Kingdom,77183.6


In [17]:
#Creating a Segmented RFM table
SegRFM = RFMtable
quantiles = SegRFM.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()

In [18]:
#RFM Basics: Low Recency, High Frequency, and High Monetary Amounts are best
#The top Recency quartile is that with the lowest recency
#Define RScore
def rscore(x,q,d):
    if x <= d[q][0.25]:
        return 1
    elif x <= d[q][0.50]:
        return 2
    elif x <=d[q][0.75]:
        return 3
    else:
        return 4
#Define Frequency and Monetary Value
#The top Frequency and Monetary quartiles are those with the highest frequency 
#and monetary values

def fmscore(x,q,d):
    if x <= d[q][0.25]:
        return 4
    elif x <= d[q][0.50]:
        return 3
    elif x <= d[q][0.75]:
        return 2
    else:
        return 1


In [19]:
SegRFM['R'] = SegRFM['recency'].apply(rscore, args=('recency',quantiles,))
SegRFM['F'] = SegRFM['frequency'].apply(fmscore, args=('frequency',quantiles,))
SegRFM['M'] = SegRFM['monetary_value'].apply(fmscore, args=('monetary_value',quantiles,))

In [20]:
SegRFM.head()

,recency,frequency,monetary_value,R,F,M
CustomerID,,,,,,
12346,346,1,77183.60,4,4,1
12354,253,58,1079.40,4,2,2
12413,87,38,758.10,3,3,2
12417,246,23,436.30,4,3,3
12421,36,45,807.04,1,2,2


In [21]:
SegRFM['RFMScore'] = SegRFM['R'].astype(str) + SegRFM['F'].astype(str) + SegRFM['M'].astype(str)
SegRFM.head()

,recency,frequency,monetary_value,R,F,M,RFMScore
CustomerID,,,,,,,
12346,346,1,77183.60,4,4,1,441
12354,253,58,1079.40,4,2,2,422
12413,87,38,758.10,3,3,2,332
12417,246,23,436.30,4,3,3,433
12421,36,45,807.04,1,2,2,122


In [22]:
SegRFM[SegRFM['RFMScore'] == '111'].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,R,F,M,RFMScore
CustomerID,,,,,,,
18102,21,431,259657.30,1,1,1,111
17450,29,337,194550.79,1,1,1,111
17511,23,963,91062.38,1,1,1,111
16684,25,277,66653.56,1,1,1,111
14096,25,5111,65164.79,1,1,1,111
13694,24,568,65039.62,1,1,1,111
15311,21,2379,60767.90,1,1,1,111
13089,23,1818,58825.83,1,1,1,111
15769,28,130,56252.72,1,1,1,111


In [23]:
#Output to CSV
SegRFM.to_csv('RFMoutput.csv')